In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
print("⚠️  Nested kernels require CUDA C++ with -rdc=true!")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

# Day 4: Nested Kernel Patterns - Russian Nesting Dolls

## 🎯 The Hook: Matryoshka Dolls

**Have you ever opened a Russian nesting doll (matryoshka)?** You open the outer doll, and inside is another doll. Open that one, and there's another. And another. Each doll contains a complete, smaller version of itself.

That's exactly how **nested kernels** work in CUDA:
- The outer kernel launches child kernels
- Those children can launch THEIR children
- Each level is a complete, functional kernel
- Up to 24 levels deep (that's a LOT of dolls!)

**Why nest kernels?**
- **Recursive algorithms**: Quicksort naturally divides and conquers
- **Adaptive detail**: Refine only where needed (like zooming into fractals)
- **Tree structures**: Each node spawns work for its children

---

## 📚 Learning Objectives

By the end of this session, you will:

1. **Implement** recursive parallel algorithms using nested kernels
2. **Design** effective base cases and recursion limits
3. **Optimize** nested launches with thresholds and batching
4. **Apply** patterns for quicksort, tree traversal, and adaptive refinement
5. **Avoid** common pitfalls (stack overflow, excessive overhead)

---

## 🪆 Concept Card: Russian Nesting Dolls (Matryoshka)

```
NESTED KERNELS = MATRYOSHKA DOLLS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Open the outer doll → Find another doll inside
Open that doll     → Another one!
Keep opening       → Until you reach the tiny solid doll

┌──────────────────────────────────────────────┐
│  Outer Kernel (Biggest Doll)                 │
│  ┌────────────────────────────────────────┐  │
│  │  Child Kernel (Medium Doll)            │  │
│  │  ┌──────────────────────────────────┐  │  │
│  │  │  Grandchild Kernel (Small Doll)  │  │  │
│  │  │  ┌────────────────────────────┐  │  │  │
│  │  │  │  Base Case (Solid Doll)   │  │  │  │
│  │  │  │  No more nesting!         │  │  │  │
│  │  │  └────────────────────────────┘  │  │  │
│  │  └──────────────────────────────────┘  │  │
│  └────────────────────────────────────────┘  │
└──────────────────────────────────────────────┘

Quicksort Example:
┌────────────────────────────────────────────────┐
│  sortKernel([8,3,7,1,5,2,9,4])                 │
│  │ pivot = 5                                   │
│  │                                             │
│  ├─→ sortKernel([3,1,2,4])    // Left doll    │
│  │   │ pivot = 2                               │
│  │   ├─→ sortKernel([1])      // Base case!   │
│  │   └─→ sortKernel([3,4])                    │
│  │       └─→ ...                               │
│  │                                             │
│  └─→ sortKernel([8,7,9])      // Right doll   │
│      └─→ ...                                   │
└────────────────────────────────────────────────┘

Key Design Patterns:
• MIN_SIZE = 32:   "Doll too small? Just sort directly!"
• MAX_DEPTH = 16:  "Don't open more than 16 dolls"
• Non-blocking:    "Siblings can open in parallel"
```

---

## Part 1: Recursive Patterns

### 🔷 CUDA C++ Parallel Quicksort (Primary)

This example demonstrates recursive quicksort using dynamic parallelism:
- Child kernels are launched from within device code
- Uses non-blocking streams for parallel left/right partitioning
- Falls back to insertion sort for small arrays (MIN_SIZE threshold)
- Limits recursion depth (MAX_DEPTH) to prevent stack overflow

**Compile with:** `nvcc -arch=sm_75 -rdc=true -o quicksort quicksort_dp.cu -lcudadevrt`

In [ ]:
%%writefile quicksort_dp.cu
// quicksort_dp.cu - Recursive quicksort with dynamic parallelism
#include <stdio.h>
#include <cuda_runtime.h>

#define MAX_DEPTH 16
#define MIN_SIZE 32  // Switch to sequential sort

// ============================================
// Sequential Sort for Small Arrays
// ============================================
__device__ void insertionSort(int* data, int left, int right) {
    for (int i = left + 1; i <= right; i++) {
        int key = data[i];
        int j = i - 1;
        while (j >= left && data[j] > key) {
            data[j + 1] = data[j];
            j--;
        }
        data[j + 1] = key;
    }
}

// ============================================
// Partition
// ============================================
__device__ int partition(int* data, int left, int right) {
    int pivot = data[right];
    int i = left - 1;
    
    for (int j = left; j < right; j++) {
        if (data[j] <= pivot) {
            i++;
            int temp = data[i];
            data[i] = data[j];
            data[j] = temp;
        }
    }
    
    int temp = data[i + 1];
    data[i + 1] = data[right];
    data[right] = temp;
    
    return i + 1;
}

// ============================================
// Recursive Quicksort Kernel
// ============================================
__global__ void quicksortKernel(int* data, int left, int right, int depth) {
    // Base case: small array or max depth
    if (right - left < MIN_SIZE || depth >= MAX_DEPTH) {
        insertionSort(data, left, right);
        return;
    }
    
    // Partition
    int pivotIdx = partition(data, left, right);
    
    // Launch child kernels for left and right partitions
    cudaStream_t s1, s2;
    cudaStreamCreateWithFlags(&s1, cudaStreamNonBlocking);
    cudaStreamCreateWithFlags(&s2, cudaStreamNonBlocking);
    
    // Left partition
    if (left < pivotIdx - 1) {
        quicksortKernel<<<1, 1, 0, s1>>>(data, left, pivotIdx - 1, depth + 1);
    }
    
    // Right partition
    if (pivotIdx + 1 < right) {
        quicksortKernel<<<1, 1, 0, s2>>>(data, pivotIdx + 1, right, depth + 1);
    }
    
    // Wait for children
    cudaDeviceSynchronize();
    cudaStreamDestroy(s1);
    cudaStreamDestroy(s2);
}

// ============================================
// Host Entry Point
// ============================================
__global__ void quicksortEntry(int* data, int n) {
    quicksortKernel<<<1, 1>>>(data, 0, n - 1, 0);
    cudaDeviceSynchronize();
}

int main() {
    const int N = 1024;
    int* h_data = new int[N];
    int* d_data;
    
    // Random data
    srand(42);
    for (int i = 0; i < N; i++) h_data[i] = rand() % 10000;
    
    printf("First 10 elements before: ");
    for (int i = 0; i < 10; i++) printf("%d ", h_data[i]);
    printf("\n");
    
    cudaMalloc(&d_data, N * sizeof(int));
    cudaMemcpy(d_data, h_data, N * sizeof(int), cudaMemcpyHostToDevice);
    
    // Sort
    quicksortEntry<<<1, 1>>>(d_data, N);
    cudaDeviceSynchronize();
    
    // Check for errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(err));
        return 1;
    }
    
    // Verify
    cudaMemcpy(h_data, d_data, N * sizeof(int), cudaMemcpyDeviceToHost);
    
    printf("First 10 elements after:  ");
    for (int i = 0; i < 10; i++) printf("%d ", h_data[i]);
    printf("\n");
    
    bool sorted = true;
    for (int i = 1; i < N; i++) {
        if (h_data[i] < h_data[i-1]) {
            sorted = false;
            break;
        }
    }
    printf("Array sorted: %s\n", sorted ? "YES" : "NO");
    
    delete[] h_data;
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -rdc=true -o quicksort_dp quicksort_dp.cu -lcudadevrt && ./quicksort_dp

---

## Part 2: Tree/Graph Traversal

### 🔷 CUDA C++ Parallel Tree Processing (Primary)

This example demonstrates parallel tree traversal using dynamic parallelism:
- Processes a binary tree structure stored in device memory
- Recursively launches child kernels for left and right subtrees
- Uses atomicAdd to accumulate values across all nodes
- Depth guard prevents infinite recursion

In [ ]:
%%writefile tree_traversal.cu
// tree_traversal.cu - Parallel tree processing
#include <stdio.h>
#include <cuda_runtime.h>

struct TreeNode {
    float value;
    int leftChild;   // Index, -1 if none
    int rightChild;  // Index, -1 if none
};

// ============================================
// Process Node and Recurse to Children
// ============================================
__global__ void processTree(TreeNode* tree, float* results, int nodeIdx, int depth) {
    if (nodeIdx < 0 || depth > 20) return;  // Guard
    
    TreeNode* node = &tree[nodeIdx];
    
    // Process this node (accumulate value)
    atomicAdd(&results[0], node->value);
    
    // Launch children in parallel
    if (node->leftChild >= 0 && node->rightChild >= 0) {
        // Both children exist - launch in parallel
        cudaStream_t s1, s2;
        cudaStreamCreateWithFlags(&s1, cudaStreamNonBlocking);
        cudaStreamCreateWithFlags(&s2, cudaStreamNonBlocking);
        
        processTree<<<1, 1, 0, s1>>>(tree, results, node->leftChild, depth + 1);
        processTree<<<1, 1, 0, s2>>>(tree, results, node->rightChild, depth + 1);
        
        cudaDeviceSynchronize();
        cudaStreamDestroy(s1);
        cudaStreamDestroy(s2);
    } else {
        // One or no children
        if (node->leftChild >= 0) {
            processTree<<<1, 1>>>(tree, results, node->leftChild, depth + 1);
            cudaDeviceSynchronize();
        }
        if (node->rightChild >= 0) {
            processTree<<<1, 1>>>(tree, results, node->rightChild, depth + 1);
            cudaDeviceSynchronize();
        }
    }
}

int main() {
    // Build simple tree:
    //       0(10)
    //      /    \
    //    1(20)  2(30)
    //    /  \
    //  3(5) 4(15)
    
    TreeNode h_tree[5];
    h_tree[0] = {10.0f, 1, 2};    // Root
    h_tree[1] = {20.0f, 3, 4};    // Left child
    h_tree[2] = {30.0f, -1, -1};  // Right child (leaf)
    h_tree[3] = {5.0f, -1, -1};   // Leaf
    h_tree[4] = {15.0f, -1, -1};  // Leaf
    
    printf("Tree structure:\n");
    printf("       0(10)\n");
    printf("      /    \\\n");
    printf("    1(20)  2(30)\n");
    printf("    /  \\\n");
    printf("  3(5) 4(15)\n\n");
    
    TreeNode* d_tree;
    float* d_result;
    
    cudaMalloc(&d_tree, 5 * sizeof(TreeNode));
    cudaMalloc(&d_result, sizeof(float));
    
    cudaMemcpy(d_tree, h_tree, 5 * sizeof(TreeNode), cudaMemcpyHostToDevice);
    cudaMemset(d_result, 0, sizeof(float));
    
    // Process tree starting at root
    processTree<<<1, 1>>>(d_tree, d_result, 0, 0);
    cudaDeviceSynchronize();
    
    // Check for errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(err));
        return 1;
    }
    
    float sum;
    cudaMemcpy(&sum, d_result, sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("Tree sum: %.0f (expected 80 = 10+20+30+5+15)\n", sum);
    
    cudaFree(d_tree);
    cudaFree(d_result);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -rdc=true -o tree_traversal tree_traversal.cu -lcudadevrt && ./tree_traversal

---

## Part 3: Adaptive Mesh Refinement

### 🔷 CUDA C++ Conditional Work Spawning (Primary)

This example demonstrates adaptive mesh refinement using dynamic parallelism:
- Cells are processed and refined based on error estimates
- Refinement creates 4 child cells (quad subdivision pattern)
- Uses atomicAdd to track dynamic cell allocation
- Recurses only when error exceeds threshold

In [ ]:
%%writefile adaptive_mesh.cu
// adaptive_mesh.cu - Adaptive refinement pattern
#include <stdio.h>
#include <cuda_runtime.h>
#include <math.h>

struct Cell {
    float value;
    float error;  // Error estimate
    int refined;  // Has been refined
};

#define ERROR_THRESHOLD 0.1f
#define MAX_DEPTH 3

// ============================================
// Process Cell - Refine if Error Too High
// ============================================
__global__ void processCell(Cell* cells, int cellIdx, int* childStart, int depth) {
    if (depth >= MAX_DEPTH) return;
    
    Cell* cell = &cells[cellIdx];
    
    // Compute work (simplified)
    cell->value = sinf((float)cellIdx * 0.5f);
    cell->error = fabsf(cell->value - 0.5f) * 0.2f;  // Fake error
    
    // Check if refinement needed
    if (cell->error > ERROR_THRESHOLD && depth < MAX_DEPTH) {
        cell->refined = 1;
        
        // Allocate child cells atomically
        int childBase = atomicAdd(childStart, 4);  // 4 children (2D quad)
        
        // Launch refinement for each child
        for (int i = 0; i < 4; i++) {
            processCell<<<1, 1>>>(cells, childBase + i, childStart, depth + 1);
        }
        
        cudaDeviceSynchronize();
    } else {
        cell->refined = 0;
    }
}

// ============================================
// Entry Kernel - Start with Coarse Grid
// ============================================
__global__ void adaptiveMesh(Cell* cells, int initialCells, int* childStart) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (tid < initialCells) {
        processCell<<<1, 1>>>(cells, tid, childStart, 0);
    }
    
    cudaDeviceSynchronize();
}

int main() {
    const int INITIAL_CELLS = 16;
    const int MAX_CELLS = 10000;  // Preallocate
    
    Cell* d_cells;
    int* d_childStart;
    
    cudaMalloc(&d_cells, MAX_CELLS * sizeof(Cell));
    cudaMalloc(&d_childStart, sizeof(int));
    
    int initialChild = INITIAL_CELLS;  // Children start after initial
    cudaMemcpy(d_childStart, &initialChild, sizeof(int), cudaMemcpyHostToDevice);
    
    printf("Starting adaptive mesh refinement...\n");
    printf("Initial cells: %d\n", INITIAL_CELLS);
    printf("Error threshold: %.2f\n", ERROR_THRESHOLD);
    printf("Max depth: %d\n\n", MAX_DEPTH);
    
    // Start adaptive refinement
    adaptiveMesh<<<1, INITIAL_CELLS>>>(d_cells, INITIAL_CELLS, d_childStart);
    cudaDeviceSynchronize();
    
    // Check for errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(err));
        return 1;
    }
    
    int totalCells;
    cudaMemcpy(&totalCells, d_childStart, sizeof(int), cudaMemcpyDeviceToHost);
    
    printf("Total cells after refinement: %d\n", totalCells);
    printf("Cells created by refinement: %d\n", totalCells - INITIAL_CELLS);
    
    cudaFree(d_cells);
    cudaFree(d_childStart);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -rdc=true -o adaptive_mesh adaptive_mesh.cu -lcudadevrt && ./adaptive_mesh

---

## Part 4: Performance Considerations

### Overhead and Optimization

```
Dynamic Parallelism Overhead:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Each child launch has overhead:
• ~5-20 μs per launch (varies)
• Memory for child kernel state
• Device runtime overhead

Optimization Strategies:
━━━━━━━━━━━━━━━━━━━━━━━

1. Batch Work
   ❌ Launch child for each element
   ✅ Launch child for chunk of elements

2. Limit Depth
   ❌ Recurse until single element
   ✅ Switch to sequential at threshold

3. Use Streams
   ❌ Serial child launches
   ✅ Parallel with non-blocking streams

4. Consider Alternatives
   • Cooperative groups for some cases
   • Flattened iteration
   • Work queues
```

### 🔷 CUDA C++ When NOT to Use Dynamic Parallelism (Primary)

```cpp
// BAD: Simple vector add - no need for DP!
__global__ void vectorAddDP(float* a, float* b, float* c, int n) {
    int tid = threadIdx.x;
    // DON'T DO THIS!
    vectorAddChild<<<1, 1>>>(a, b, c, tid);
}

// GOOD: Just use regular parallelism
__global__ void vectorAdd(float* a, float* b, float* c, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) c[tid] = a[tid] + b[tid];
}
```

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile nested_kernels_exercises.cu
// nested_kernels_exercises.cu - Nested Kernel Pattern Exercises
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CHECK_CUDA(call) { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, \
               cudaGetErrorString(err)); \
        exit(1); \
    } \
}

// ============================================
// Exercise 1: Parallel Merge Sort with DP
// ============================================
#define MIN_SIZE 32  // Switch to insertion sort

__device__ void insertionSort(int* arr, int left, int right) {
    for (int i = left + 1; i <= right; i++) {
        int key = arr[i];
        int j = i - 1;
        while (j >= left && arr[j] > key) {
            arr[j + 1] = arr[j];
            j--;
        }
        arr[j + 1] = key;
    }
}

__device__ void merge(int* arr, int left, int mid, int right, int* temp) {
    int i = left, j = mid + 1, k = left;
    
    while (i <= mid && j <= right) {
        if (arr[i] <= arr[j]) {
            temp[k++] = arr[i++];
        } else {
            temp[k++] = arr[j++];
        }
    }
    while (i <= mid) temp[k++] = arr[i++];
    while (j <= right) temp[k++] = arr[j++];
    
    for (int l = left; l <= right; l++) {
        arr[l] = temp[l];
    }
}

__global__ void mergeSortKernel(int* arr, int* temp, int left, int right, int depth);

__global__ void mergeSortKernel(int* arr, int* temp, int left, int right, int depth) {
    if (right - left < MIN_SIZE) {
        insertionSort(arr, left, right);
        return;
    }
    
    int mid = left + (right - left) / 2;
    
    if (depth > 0) {
        cudaStream_t s1, s2;
        cudaStreamCreateWithFlags(&s1, cudaStreamNonBlocking);
        cudaStreamCreateWithFlags(&s2, cudaStreamNonBlocking);
        
        // Launch child kernels for left and right halves
        mergeSortKernel<<<1, 1, 0, s1>>>(arr, temp, left, mid, depth - 1);
        mergeSortKernel<<<1, 1, 0, s2>>>(arr, temp, mid + 1, right, depth - 1);
        
        cudaDeviceSynchronize();
        
        cudaStreamDestroy(s1);
        cudaStreamDestroy(s2);
    } else {
        // Sequential recursion at max depth
        insertionSort(arr, left, mid);
        insertionSort(arr, mid + 1, right);
    }
    
    merge(arr, left, mid, right, temp);
}

void testMergeSort() {
    printf("\n=== Exercise 1: Parallel Merge Sort ===\n");
    
    const int N = 1024;
    int* h_arr = (int*)malloc(N * sizeof(int));
    
    // Initialize with random values
    srand(42);
    for (int i = 0; i < N; i++) {
        h_arr[i] = rand() % 10000;
    }
    
    int *d_arr, *d_temp;
    CHECK_CUDA(cudaMalloc(&d_arr, N * sizeof(int)));
    CHECK_CUDA(cudaMalloc(&d_temp, N * sizeof(int)));
    CHECK_CUDA(cudaMemcpy(d_arr, h_arr, N * sizeof(int), cudaMemcpyHostToDevice));
    
    printf("Sorting %d elements with nested merge sort...\n", N);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    mergeSortKernel<<<1, 1>>>(d_arr, d_temp, 0, N - 1, 4);  // depth = 4
    cudaEventRecord(stop);
    
    CHECK_CUDA(cudaDeviceSynchronize());
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    CHECK_CUDA(cudaMemcpy(h_arr, d_arr, N * sizeof(int), cudaMemcpyDeviceToHost));
    
    // Verify sorted
    bool sorted = true;
    for (int i = 1; i < N; i++) {
        if (h_arr[i] < h_arr[i-1]) {
            sorted = false;
            break;
        }
    }
    
    printf("Result: %s\n", sorted ? "SORTED ✓" : "NOT SORTED ✗");
    printf("Time: %.3f ms\n", ms);
    printf("First 10: ");
    for (int i = 0; i < 10; i++) printf("%d ", h_arr[i]);
    printf("\n");
    
    cudaFree(d_arr);
    cudaFree(d_temp);
    free(h_arr);
}

// ============================================
// Exercise 2: Adaptive Quadtree Construction
// ============================================
struct Point { float x, y; };
struct BBox { float xmin, ymin, xmax, ymax; };

#define MAX_POINTS_PER_NODE 4
#define MAX_DEPTH 6

__device__ bool inBox(Point p, BBox box) {
    return p.x >= box.xmin && p.x < box.xmax &&
           p.y >= box.ymin && p.y < box.ymax;
}

__global__ void quadtreeKernel(Point* points, int* indices, int n,
                                BBox box, int depth, int* nodeCount) {
    if (n <= MAX_POINTS_PER_NODE || depth >= MAX_DEPTH) {
        atomicAdd(nodeCount, 1);  // Count leaf node
        if (threadIdx.x == 0) {
            printf("Leaf at depth %d: %d points in [%.1f,%.1f]-[%.1f,%.1f]\n",
                   depth, n, box.xmin, box.ymin, box.xmax, box.ymax);
        }
        return;
    }
    
    float midX = (box.xmin + box.xmax) / 2;
    float midY = (box.ymin + box.ymax) / 2;
    
    // Count points in each quadrant
    int counts[4] = {0, 0, 0, 0};
    BBox quadrants[4] = {
        {box.xmin, box.ymin, midX, midY},       // SW
        {midX, box.ymin, box.xmax, midY},       // SE
        {box.xmin, midY, midX, box.ymax},       // NW
        {midX, midY, box.xmax, box.ymax}        // NE
    };
    
    for (int i = 0; i < n; i++) {
        Point p = points[indices[i]];
        for (int q = 0; q < 4; q++) {
            if (inBox(p, quadrants[q])) {
                counts[q]++;
                break;
            }
        }
    }
    
    // Launch child kernels for non-empty quadrants
    for (int q = 0; q < 4; q++) {
        if (counts[q] > 0) {
            // Allocate child indices (simplified - in production use proper allocation)
            quadtreeKernel<<<1, 1>>>(points, indices, counts[q],
                                      quadrants[q], depth + 1, nodeCount);
        }
    }
    cudaDeviceSynchronize();
}

void testQuadtree() {
    printf("\n=== Exercise 2: Adaptive Quadtree ===\n");
    
    const int N = 64;
    Point* h_points = (Point*)malloc(N * sizeof(Point));
    int* h_indices = (int*)malloc(N * sizeof(int));
    
    // Create clustered point distribution
    srand(42);
    for (int i = 0; i < N; i++) {
        // Cluster points in different regions
        int cluster = i % 4;
        float cx = (cluster % 2) * 0.5f + 0.25f;
        float cy = (cluster / 2) * 0.5f + 0.25f;
        h_points[i].x = cx + (rand() % 100) / 1000.0f;
        h_points[i].y = cy + (rand() % 100) / 1000.0f;
        h_indices[i] = i;
    }
    
    Point *d_points;
    int *d_indices, *d_nodeCount;
    CHECK_CUDA(cudaMalloc(&d_points, N * sizeof(Point)));
    CHECK_CUDA(cudaMalloc(&d_indices, N * sizeof(int)));
    CHECK_CUDA(cudaMalloc(&d_nodeCount, sizeof(int)));
    CHECK_CUDA(cudaMemcpy(d_points, h_points, N * sizeof(Point), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_indices, h_indices, N * sizeof(int), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemset(d_nodeCount, 0, sizeof(int)));
    
    BBox rootBox = {0.0f, 0.0f, 1.0f, 1.0f};
    
    printf("Building quadtree for %d points...\n\n", N);
    quadtreeKernel<<<1, 1>>>(d_points, d_indices, N, rootBox, 0, d_nodeCount);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    int nodeCount;
    CHECK_CUDA(cudaMemcpy(&nodeCount, d_nodeCount, sizeof(int), cudaMemcpyDeviceToHost));
    printf("\nTotal leaf nodes created: %d\n", nodeCount);
    
    cudaFree(d_points);
    cudaFree(d_indices);
    cudaFree(d_nodeCount);
    free(h_points);
    free(h_indices);
}

// ============================================
// Exercise 3: Adaptive Mandelbrot Refinement
// ============================================
__device__ int mandelbrot(float cx, float cy, int maxIter) {
    float zx = 0, zy = 0;
    int iter = 0;
    while (zx*zx + zy*zy < 4.0f && iter < maxIter) {
        float tmp = zx*zx - zy*zy + cx;
        zy = 2*zx*zy + cy;
        zx = tmp;
        iter++;
    }
    return iter;
}

#define REFINE_THRESHOLD 0.3f  // Variance threshold for refinement
#define MIN_CELL_SIZE 4

__global__ void mandelbrotRefineKernel(int* output, int width, int height,
                                        int x, int y, int cellSize,
                                        float xmin, float xmax,
                                        float ymin, float ymax,
                                        int maxIter, int depth) {
    if (cellSize <= MIN_CELL_SIZE || depth > 4) {
        // Compute all pixels in this cell
        for (int py = y; py < y + cellSize && py < height; py++) {
            for (int px = x; px < x + cellSize && px < width; px++) {
                float cx = xmin + (xmax - xmin) * px / width;
                float cy = ymin + (ymax - ymin) * py / height;
                output[py * width + px] = mandelbrot(cx, cy, maxIter);
            }
        }
        return;
    }
    
    // Sample corners and center
    float corners[5][2] = {
        {(float)x, (float)y},
        {(float)(x + cellSize - 1), (float)y},
        {(float)x, (float)(y + cellSize - 1)},
        {(float)(x + cellSize - 1), (float)(y + cellSize - 1)},
        {(float)(x + cellSize/2), (float)(y + cellSize/2)}
    };
    
    int values[5];
    float sum = 0, sumSq = 0;
    for (int i = 0; i < 5; i++) {
        float cx = xmin + (xmax - xmin) * corners[i][0] / width;
        float cy = ymin + (ymax - ymin) * corners[i][1] / height;
        values[i] = mandelbrot(cx, cy, maxIter);
        float v = (float)values[i] / maxIter;
        sum += v;
        sumSq += v * v;
    }
    
    float mean = sum / 5;
    float variance = sumSq / 5 - mean * mean;
    
    if (variance > REFINE_THRESHOLD * REFINE_THRESHOLD) {
        // High variance - subdivide
        int half = cellSize / 2;
        cudaStream_t s1, s2, s3, s4;
        cudaStreamCreateWithFlags(&s1, cudaStreamNonBlocking);
        cudaStreamCreateWithFlags(&s2, cudaStreamNonBlocking);
        cudaStreamCreateWithFlags(&s3, cudaStreamNonBlocking);
        cudaStreamCreateWithFlags(&s4, cudaStreamNonBlocking);
        
        mandelbrotRefineKernel<<<1, 1, 0, s1>>>(output, width, height,
            x, y, half, xmin, xmax, ymin, ymax, maxIter, depth + 1);
        mandelbrotRefineKernel<<<1, 1, 0, s2>>>(output, width, height,
            x + half, y, half, xmin, xmax, ymin, ymax, maxIter, depth + 1);
        mandelbrotRefineKernel<<<1, 1, 0, s3>>>(output, width, height,
            x, y + half, half, xmin, xmax, ymin, ymax, maxIter, depth + 1);
        mandelbrotRefineKernel<<<1, 1, 0, s4>>>(output, width, height,
            x + half, y + half, half, xmin, xmax, ymin, ymax, maxIter, depth + 1);
        
        cudaDeviceSynchronize();
        
        cudaStreamDestroy(s1);
        cudaStreamDestroy(s2);
        cudaStreamDestroy(s3);
        cudaStreamDestroy(s4);
    } else {
        // Low variance - fill with average value
        int avg = (int)(mean * maxIter);
        for (int py = y; py < y + cellSize && py < height; py++) {
            for (int px = x; px < x + cellSize && px < width; px++) {
                output[py * width + px] = avg;
            }
        }
    }
}

void testMandelbrot() {
    printf("\n=== Exercise 3: Adaptive Mandelbrot ===\n");
    
    const int W = 128, H = 128;
    const int maxIter = 256;
    
    int* d_output;
    CHECK_CUDA(cudaMalloc(&d_output, W * H * sizeof(int)));
    CHECK_CUDA(cudaMemset(d_output, 0, W * H * sizeof(int)));
    
    printf("Rendering %dx%d Mandelbrot with adaptive refinement...\n", W, H);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    
    // Launch for 4 initial quadrants
    int half = W / 2;
    cudaStream_t streams[4];
    for (int i = 0; i < 4; i++) {
        cudaStreamCreate(&streams[i]);
    }
    
    mandelbrotRefineKernel<<<1, 1, 0, streams[0]>>>(d_output, W, H, 0, 0, half,
        -2.0f, 1.0f, -1.5f, 1.5f, maxIter, 0);
    mandelbrotRefineKernel<<<1, 1, 0, streams[1]>>>(d_output, W, H, half, 0, half,
        -2.0f, 1.0f, -1.5f, 1.5f, maxIter, 0);
    mandelbrotRefineKernel<<<1, 1, 0, streams[2]>>>(d_output, W, H, 0, half, half,
        -2.0f, 1.0f, -1.5f, 1.5f, maxIter, 0);
    mandelbrotRefineKernel<<<1, 1, 0, streams[3]>>>(d_output, W, H, half, half, half,
        -2.0f, 1.0f, -1.5f, 1.5f, maxIter, 0);
    
    cudaEventRecord(stop);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    int* h_output = (int*)malloc(W * H * sizeof(int));
    CHECK_CUDA(cudaMemcpy(h_output, d_output, W * H * sizeof(int), cudaMemcpyDeviceToHost));
    
    // Count non-zero pixels (computed regions)
    int computed = 0;
    for (int i = 0; i < W * H; i++) {
        if (h_output[i] > 0) computed++;
    }
    
    printf("Render time: %.3f ms\n", ms);
    printf("Computed pixels: %d/%d (%.1f%%)\n", 
           computed, W * H, 100.0f * computed / (W * H));
    
    // Show small ASCII visualization
    printf("\nAscii preview (16x8):\n");
    const char* chars = " .:-=+*#%@";
    for (int y = 0; y < H; y += H/8) {
        for (int x = 0; x < W; x += W/16) {
            int val = h_output[y * W + x];
            int idx = (val * 9) / maxIter;
            printf("%c", chars[idx < 10 ? idx : 9]);
        }
        printf("\n");
    }
    
    for (int i = 0; i < 4; i++) cudaStreamDestroy(streams[i]);
    cudaFree(d_output);
    free(h_output);
}

// ============================================
// Main
// ============================================
int main() {
    printf("╔══════════════════════════════════════════════════════════════╗\n");
    printf("║         NESTED KERNEL PATTERNS - EXERCISES                    ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n");
    
    int device;
    cudaDeviceProp prop;
    cudaGetDevice(&device);
    cudaGetDeviceProperties(&prop, device);
    printf("\nDevice: %s (Compute %d.%d)\n", prop.name, prop.major, prop.minor);
    
    testMergeSort();
    testQuadtree();
    testMandelbrot();
    
    printf("\n✓ All exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -rdc=true -lcudadevrt -o nested_kernels_exercises nested_kernels_exercises.cu && ./nested_kernels_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: Merge Sort
Implement parallel merge sort using dynamic parallelism.

### Exercise 2: Quadtree Construction
Build a quadtree for 2D points using adaptive subdivision.

### Exercise 3: Fractal Rendering
Use DP to adaptively refine Mandelbrot set regions with high detail.

---

## Key Takeaways

```
┌─────────────────────────────────────────────────────────┐
│          NESTED KERNELS - MATRYOSHKA PATTERNS           │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  🪆 Matryoshka Analogy:                                 │
│  • Each kernel is a complete doll containing others     │
│  • Keep opening until you hit the solid base case       │
│  • Each level is independent and self-contained         │
│  • Don't nest too deep (24 levels max, 16 recommended)  │
│                                                         │
├─────────────────────────────────────────────────────────┤
│  Design Patterns:                                       │
│                                                         │
│  1. Base Case (Solid Doll):                             │
│     if (size < MIN_SIZE) {                              │
│         insertionSort();  // No more nesting!           │
│         return;                                         │
│     }                                                   │
│                                                         │
│  2. Depth Limit:                                        │
│     if (depth >= MAX_DEPTH) {                           │
│         serialProcess();  // Stop recursing             │
│         return;                                         │
│     }                                                   │
│                                                         │
│  3. Parallel Siblings:                                  │
│     cudaStream_t left, right;                           │
│     kernel<<<g,b,0,left>>>();   // Parallel!            │
│     kernel<<<g,b,0,right>>>();  // Parallel!            │
│                                                         │
├─────────────────────────────────────────────────────────┤
│  Good Use Cases:                                        │
│  ✓ Recursive algorithms (sort, tree traversal)          │
│  ✓ Adaptive refinement (fractals, mesh)                 │
│  ✓ Data-dependent parallelism (sparse)                  │
│                                                         │
│  Avoid When:                                            │
│  ✗ Regular parallelism works fine                       │
│  ✗ Work is uniform/predictable                          │
│  ✗ Launch overhead dominates compute                    │
│                                                         │
│  Memory: Only global memory shared between dolls!       │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

---

## 🎉 Week 11 Complete!

**Congratulations!** You've mastered the advanced cooperative programming patterns:

| Day | Topic | Analogy | Key Skill |
|-----|-------|---------|-----------|
| 1 | Cooperative Groups | 🏀 Basketball team plays | Thread coordination at any level |
| 2 | Grid Sync | 🏟️ Stadium wave | All-blocks synchronization |
| 3 | Dynamic Parallelism | 👔 Manager delegation | Kernels launching kernels |
| 4 | Nested Kernels | 🪆 Matryoshka dolls | Recursive parallel patterns |

---

## 🔗 What's Next?

**Week 12: Multi-GPU & Advanced Topics** - You've mastered single-GPU cooperative patterns. Now it's time to scale to MULTIPLE GPUs!

You'll learn:
- Multi-GPU programming with peer-to-peer access
- CUDA streams for maximum concurrency
- NVTX profiling for complex applications
- Advanced optimization techniques

*From matryoshka dolls to a whole doll factory running in parallel!*